In [1]:
import pandas as pd
from Models.WhiteBox.figs.figs_class import FIGS_class
%reload_ext autoreload
%autoreload 2

In [2]:
FIGS_parameter = dict(
    max_rules= [5,10,15,20,25],
    max_trees=[2,3,4,5,6,7,8,9,10],
    min_impurity_decrease=[0.1, 0.2, 0.3],
)
gridsearch_parameters = dict(
    path="./../../../outputs/scores/",
    name="0",
    nDCG_at=15
)

# Grid-search

In [3]:
gs = FIGS_class(**gridsearch_parameters)

In [4]:
best_ = gs.grid_search(FIGS_parameter)
gs.save_model(best_[0], name="FIGSReg")

Finding the best model: 100%|██████████| 135/135 [01:50<00:00,  1.23it/s, nDCG=0.98]


# Metric evalutation

In [5]:
best_model = gs.load_model(name="./saved_models/FIGSReg")

In [6]:
# nDCG on test-set
nDCG_train = gs.eval_model(model=best_model, df=gs.train, nDCG_at=[1,10,15])
nDCG_valid = gs.eval_model(model=best_model, df=gs.valid, nDCG_at=[1,10,15])
nDCG_test = gs.eval_model(model=best_model, df=gs.test, nDCG_at=[1,10,15])

display(pd.DataFrame([nDCG_train,nDCG_valid,nDCG_test],index=["Training","Validation","Test"]))

,nDCG@1,nDCG@10,nDCG@15
Training,0.9160,0.9528,0.9626
Validation,0.9296,0.9713,0.9801
Test,0.9298,0.9600,0.9703


In [7]:
print(best_model)

> ------------------------------
> FIGS-Fast Interpretable Greedy-Tree Sums:
> 	Predictions are made by summing the "Val" reached by traversing each tree.
> 	For classifiers, a sigmoid function is then applied to the sum.
> ------------------------------
fitness_know_essential <= 0.125 (Tree #0 root)
	fitness_edu_basic <= 0.500 (split)
		Val: 26.531 (leaf)
		Val: 34.384 (leaf)
	fitness_lang_lvl_basic <= 0.750 (split)
		fitness_comp_essential <= 0.417 (split)
			Val: 50.491 (leaf)
			Val: 54.059 (leaf)
		fitness_comp_essential <= 0.292 (split)
			Val: 56.276 (leaf)
			Val: 59.295 (leaf)

	+
fitness_exp_basic <= 0.500 (Tree #1 root)
	Val: -3.440 (leaf)
	fitness_exp_bonus <= 0.500 (split)
		Val: 4.579 (leaf)
		Val: 7.535 (leaf)

	+
fitness_lang_basic <= 0.250 (Tree #2 root)
	Val: -7.191 (leaf)
	fitness_comp_essential <= 0.583 (split)
		Val: 0.239 (leaf)
		Val: 3.135 (leaf)

	+
fitness_edu_bonus <= 0.500 (Tree #3 root)
	fitness_comp_sim_bonus <= 0.125 (split)
		Val: -0.563 (leaf)
		Val: 0.

In [8]:
# for i in range(len(best_model.trees_)):
#     best_model.plot(tree_number=i)